In [1]:
import geoutils as gu
import numpy as np
import xdem
import os
import shutil
import xarray as xr
import rioxarray as rioxr
import geopandas as gpd
import matplotlib.pyplot as plt
import proplot as pplt
from xdem.spatialstats import nmad
import pandas as pd
from matplotlib.ticker import FuncFormatter
import gc

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pth_vgm='D:\\Key_works\\20231212dh_calculate\\xnhl_test\\spatial_corr_analysis\\vgm_params'
pth_err='D:\\Key_works\\20231212dh_calculate\\xnhl_test\\group_err'
pth_gla='D:\\Key_works\\20231212dh_calculate\\gla\\CGI1'

### test for one group

In [19]:
pth1=pth_err+'\\g0001_heteroscedasticity.tif'
pth2=pth_vgm+'\\g0001_corr.csv'
pth3=pth_gla+'\\g0001_gla.shp'

In [86]:
params_variogram_model=pd.read_csv(pth2,index_col=0)
errors=xdem.DEM(pth1)
glacier_outlines = gpd.read_file(pth3)

merged_gdf = gpd.GeoDataFrame(geometry=[glacier_outlines.geometry.unary_union], crs=glacier_outlines.crs)
#mask_glacier = glacier_outlines.create_mask(dh)

stderr_glaciers = xdem.spatialstats.spatial_error_propagation(
    areas=[merged_gdf], errors=errors, params_variogram_model=params_variogram_model
)

stderr_glaciers1 = xdem.spatialstats.spatial_error_propagation(
    areas=merged_gdf.geometry.area.values, errors=errors, params_variogram_model=params_variogram_model
)

In [87]:
stderr_glaciers,stderr_glaciers1

([3.5554939606581177], [3.5115590323268253])

### all

In [4]:
glst=[i[0:5] for i in os.listdir(pth_vgm)]
len(glst)

261

In [5]:
#out_lst=[]
for gpi in glst:
    out_lst=[]
    if os.path.exists('tmp_file\\'+gpi+'.csv'):
        continue
    print(gpi,end='')
    parai=pd.read_csv(pth_vgm+'\\'+gpi+'_corr.csv',index_col=0)
    erri=xdem.DEM(pth_err+'\\'+gpi+'_heteroscedasticity.tif')
    gla_gpi=gpd.read_file(pth_gla+'\\'+gpi+'_gla.shp',index_col=0)
    merged_gdf = gpd.GeoDataFrame(geometry=[gla_gpi.geometry.unary_union], crs=gla_gpi.crs)

    try:
        sign='poly'
        stderr_glaciers = xdem.spatialstats.spatial_error_propagation(
            areas=[merged_gdf], errors=erri, params_variogram_model=parai)
    except:
        sign='value'
        stderr_glaciers = xdem.spatialstats.spatial_error_propagation(
            areas=merged_gdf.geometry.area.values, errors=erri, params_variogram_model=parai)
    out_lst.append([gpi,stderr_glaciers[0],sign])
    dfi=pd.DataFrame(out_lst,columns=['group','error_1sigma','calculation_form'])
    dfi.to_csv('tmp_file\\'+gpi+'.csv')
    print(' done!')
    gc.collect()

g0423 done!
g0424 done!


In [6]:
flst=[i for i in os.listdir('tmp_file') if i[-4:]=='.csv']
dfflst=[]
for nm in flst:
    dffi=pd.read_csv('tmp_file\\'+nm)
    dfflst.append(dffi)
df_all=pd.concat(dfflst)

In [18]:
df_all=df_all.drop_duplicates()[['group','error_1sigma','calculation_form']]

In [19]:
df_all.to_csv('D:\\Key_works\\20231212dh_calculate\\xnhl_test\\dh_error_propagation.csv')

In [ ]:
df_all=pd.DataFrame(out_lst,columns=['group','error_1sigma','calculation_form'])

In [23]:
len(df_all[df_all['calculation_form']=='poly'])

203

In [24]:
df_all.head()

,group,error_1sigma,calculation_form
0,g0001,3.551806,poly
1,g0002,1.934397,poly
2,g0004,4.056870,poly
3,g0005,2.301977,poly
4,g0006,7.864847,poly
